#Setup & Import

In [ ]:
pathName      = "/content/drive/My Drive/" 
pathNameData  = pathName + "Datasets/" 
pathNameTrain = pathName + "DataTrain/"
prefix        = '_2022_'


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding, Bidirectional, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


#NER

In [ ]:
df = pd.read_csv(pathNameData + 'ner.csv', encoding = "ISO-8859-1", error_bad_lines=False, index_col=0)

b'Skipping line 281837: expected 25 fields, saw 34\n'
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
df.describe()

,sentence_idx
count,1.050794e+06
mean,1.898184e+04
std,1.576237e+04
min,1.000000e+00
25%,5.997000e+03
50%,1.201700e+04
75%,3.592600e+04
max,4.795900e+04


In [ ]:
df = df.drop_duplicates()

In [ ]:
df.describe()

,sentence_idx
count,768955.000000
mean,23580.065651
std,15986.777037
min,1.000000
25%,8806.000000
50%,17579.000000
75%,39135.000000
max,47959.000000


In [ ]:
df.head(30)

,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O
5,through,london,to,TO,lowercase,to,NNP,capitalized,London,IN,O,march,VBN,O,have,VBP,lowercase,have,lowercase,marched,1.0,lowercase,through,O
6,london,to,protest,VB,lowercase,protest,TO,lowercase,to,NNP,O,through,IN,O,march,VBN,lowercase,marched,lowercase,through,1.0,capitalized,London,B-geo
7,to,protest,the,DT,lowercase,the,VB,lowercase,protest,TO,B-geo,london,NNP,O,through,IN,lowercase,through,capitalized,London,1.0,lowercase,to,O
8,protest,the,war,NN,lowercase,war,DT,lowercase,the,VB,O,to,TO,B-geo,london,NNP,capitalized,London,lowercase,to,1.0,lowercase,protest,O
9,the,war,in,IN,lowercase,in,NN,lowercase,war,DT,O,protest,VB,O,to,TO,lowercase,to,lowercase,protest,1.0,lowercase,the,O


In [ ]:
df = df[['sentence_idx', 'word','tag']]

In [ ]:
df.head(25)

,sentence_idx,word,tag
0,1.0,Thousands,O
1,1.0,of,O
2,1.0,demonstrators,O
3,1.0,have,O
4,1.0,marched,O
5,1.0,through,O
6,1.0,London,B-geo
7,1.0,to,O
8,1.0,protest,O
9,1.0,the,O


In [ ]:
df.describe()

,sentence_idx
count,768955.000000
mean,23580.065651
std,15986.777037
min,1.000000
25%,8806.000000
50%,17579.000000
75%,39135.000000
max,47959.000000


In [ ]:
max_idx = int(df.sentence_idx.max())
max_idx

47959

In [ ]:
df.loc[df['sentence_idx']==1, 'word']

0         Thousands
1                of
2     demonstrators
3              have
4           marched
5           through
6            London
7                to
8           protest
9               the
10              war
11               in
12             Iraq
13              and
14           demand
15              the
16       withdrawal
17               of
18          British
19           troops
20             from
21             that
22          country
23                .
Name: word, dtype: object

In [ ]:
sequences = []
tags = []

In [ ]:
%%time
for i in range(1, max_idx+1):
  sequences.append(df.loc[df['sentence_idx']==i, 'word'].values.tolist())
  tags.append(df.loc[df['sentence_idx']==i, 'tag'].values.tolist())

CPU times: user 3min 34s, sys: 5.58 s, total: 3min 40s
Wall time: 2min 32s


In [ ]:
print(sequences[0])
print(tags[0])

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(sequences[1])
print(tags[1])

['Families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogans', 'as', '"', 'Bush', 'Number', 'One', 'Terrorist', '"', 'and', '"', 'Stop', 'the', 'Bombings', '.', '"']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
print(sequences[2])
print(tags[2])

['They', 'marched', 'from', 'the', 'Houses', 'of', 'Parliament', 'to', 'a', 'rally', 'in', 'Hyde', 'Park', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'I-geo', 'O']


Комментарий: немного тегов-то

In [ ]:
sequences_ = [' '.join(sequence) for sequence in sequences]

In [ ]:
sequences_[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
num_words = 5000
sent_len = 20
tokenizer = Tokenizer(num_words)

In [ ]:
tokenizer.fit_on_texts(sequences_)

In [ ]:
tokenizer.index_word

{1: 'the',
 2: 'in',
 3: 'of',
 4: 'to',
 5: 'a',
 6: 'and',
 7: "'s",
 8: 'for',
 9: 'has',
 10: 'on',
 11: 'is',
 12: 'that',
 13: 'have',
 14: 'u',
 15: 'with',
 16: 'said',
 17: 'was',
 18: 'at',
 19: 'says',
 20: 's',
 21: 'from',
 22: 'by',
 23: 'he',
 24: 'an',
 25: 'as',
 26: 'say',
 27: 'it',
 28: 'are',
 29: 'were',
 30: 'his',
 31: 'officials',
 32: 'president',
 33: 'will',
 34: 'government',
 35: 'mr',
 36: 'two',
 37: 'been',
 38: 'killed',
 39: 'people',
 40: 'not',
 41: 'after',
 42: 'its',
 43: 'but',
 44: 'be',
 45: 'they',
 46: 'more',
 47: 'also',
 48: 'year',
 49: 'new',
 50: 'united',
 51: 'military',
 52: 'last',
 53: 'who',
 54: 'country',
 55: 'police',
 56: 'than',
 57: 'one',
 58: 'minister',
 59: 'iraq',
 60: 'their',
 61: 'security',
 62: 'which',
 63: 'this',
 64: 'about',
 65: 'other',
 66: 'states',
 67: 'had',
 68: 'least',
 69: 'state',
 70: 'three',
 71: 'tuesday',
 72: 'group',
 73: 'iran',
 74: 'forces',
 75: 'week',
 76: 'world',
 77: 'since',
 78:

In [ ]:
X = tokenizer.texts_to_sequences(sequences_)

In [ ]:
len(X), len(X[0]), X[0][:10], len(X[1]), len(X[10])

(47959, 23, [262, 3, 977, 13, 1958, 248, 428, 4, 523, 1], 24, 11)

In [ ]:
X = pad_sequences(X, sent_len)

In [ ]:
X.shape

(47959, 20)

In [ ]:
X[0][:15], X[1][:25]

(array([  13, 1958,  248,  428,    4,  523,    1,  122,    2,   59,    6,
         603,    1,  814,    3], dtype=int32),
 array([   2,    1,  482, 1157,    1,  655,   53,  573,   15,  308, 2743,
          25,  119,  279,   57,  331,    6,  400,    1,  498], dtype=int32))

In [ ]:
tags = [' '.join(tag) for tag in tags]

In [ ]:
len(tags), len(tags[0]), tags[0][:10], len(tags[1])

(47959, 59, 'O O O O O ', 63)

In [ ]:
tag_tokenizer = Tokenizer(df.tag.nunique(), filters=' ')

In [ ]:
tag_tokenizer.fit_on_texts(tags)

In [ ]:
tag_tokenizer.index_word

{1: 'o',
 2: 'b-geo',
 3: 'b-tim',
 4: 'b-org',
 5: 'i-per',
 6: 'b-per',
 7: 'i-org',
 8: 'b-gpe',
 9: 'i-geo',
 10: 'i-tim',
 11: 'b-art',
 12: 'b-eve',
 13: 'i-eve',
 14: 'i-art',
 15: 'b-nat',
 16: 'i-gpe',
 17: 'i-nat'}

In [ ]:
Y = tag_tokenizer.texts_to_sequences(tags)
Y = pad_sequences(Y, sent_len, value=1)

In [ ]:
Y.shape

(47959, 20)

In [ ]:
Y = to_categorical(Y, df.tag.nunique()+1)

In [ ]:
Y.shape

(47959, 20, 18)

In [ ]:
Y = Y[:, :, 1:]
Y.shape

(47959, 20, 17)

In [ ]:
df.tag.nunique()

17

In [ ]:
!pip install tf2crf


     |████████████████████████████████| 1.1 MB 6.7 MB/s 


In [ ]:
Y = tag_tokenizer.texts_to_sequences(tags)
Y = pad_sequences(Y, sent_len, value=1)
# иначе AssertionError       при обучении


In [ ]:
from tf2crf import CRF, ModelWithCRFLoss
# https://github.com/xuxingya/tf2crf
emb_size = 200

input = Input(shape=(None,))
x = Embedding(num_words, emb_size)(input)

x = Bidirectional(LSTM(emb_size, return_sequences=True))(x)
x = LSTM(emb_size * 2, return_sequences=True)(x)
x = Dense(df.tag.nunique(), activation='softmax')(x) # так тоже работает (без разбора вариант)
#x = Dense(df.tag.nunique(), activation=None)(x) # ставим activation=None согласно рекомендациям в документации 

crf = CRF()
output = crf(x)
base_model = Model(input, output)
model = ModelWithCRFLoss(base_model, sparse_target=True)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X, Y, validation_split=0.2, epochs=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


1199/1199 [==============================] - 82s 64ms/step - loss: 22.1870 - accuracy: 0.9140 - val_loss_val: 13.2414 - val_val_accuracy: 0.8769


In [ ]:
model.save_weights(prefix +'ner.h5')

In [ ]:
print(X.shape, Y.shape)
id = np.random.randint(X.shape[0])
id2 = np.random.randint(20)
pred = model.predict([X[id]])
print(f"id = {id}, id2 = {id2}", pred.shape)
print( pred[id2], "\n", np.argmax(pred[id2][0]), "\n", Y[id][id2], "\n", np.argmax(Y[id][id2]))
print(tag_tokenizer.index_word[np.argmax(Y[id][id2])+1])
#print("\n", pred.shape, pred[0], np.argmax(pred[0][0]), Y[0][0], np.argmax(Y[0][0]), tag_tokenizer.index_word[np.argmax(Y[0][0])+1])

(47959, 20) (47959, 20, 17)
id = 19893, id2 = 3 (20, 1, 17)
[[0.20714018 0.07560933 0.04837987 0.0857718  0.05511244 0.0757753
  0.05702627 0.0784815  0.04601973 0.03348988 0.03890512 0.03319428
  0.0306735  0.0370234  0.03487116 0.03378156 0.02874463]] 
 0 
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 0
o


In [ ]:
print(sequences[0])
print(tags[0])

['Thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'London', 'to', 'protest', 'the', 'war', 'in', 'Iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'British', 'troops', 'from', 'that', 'country', '.']
O O O O O O B-geo O O O O O B-geo O O O O O B-gpe O O O O O


In [ ]:
# т.к. часто попадает "О" сделаем то что знаем
id = 0
id2 = 8
pred = model.predict([X[id]])
print(f"id = {id}, id2 = {id2}", pred.shape)
print( pred[id2], "\n", np.argmax(pred[id2][0]), "\n", Y[id][id2], "\n", np.argmax(Y[id][id2]))
print(tag_tokenizer.index_word[np.argmax(Y[id][id2])+1])

id = 0, id2 = 8 (20, 1, 17)
[[0.22080643 0.23151492 0.06128117 0.07524778 0.02671159 0.03420546
  0.03686262 0.05845764 0.08217815 0.03200787 0.0204308  0.02035932
  0.02131826 0.01838454 0.01949503 0.02250039 0.01823807]] 
 1 
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 
 1
b-geo
